In [3]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import joblib
import pandas as pd
import torch
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
test_df  = pd.read_csv("../dataset/test.csv",  index_col="PassengerId")

In [7]:
drop_cols = ["Name", "Ticket", "Cabin"]

def drop_unneeded_cols_df(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(columns=drop_cols, errors="ignore")

def log1p_fare_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["Fare"] = np.log1p(df["Fare"])
    return df

# 2. NOW you can safely load the pipeline
loaded_preprocess = joblib.load("../models/titanic_preprocessor.joblib")

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size=8, num_layers=1, dropout=0.1):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, int(hidden_size))
        self.fc3 = nn.Linear(int(hidden_size), 1)
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x):
        x = torch.relu(self.fc1(x)) # We are using ReLu activation function. Without an activation function, our model would be linear. As you can see, ReLU does not allow negative values
        x = self.dropout(x)
        for _ in range(1, self.num_layers):
            x = torch.relu(self.fc2(x))
            x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
# 1. Recreate the empty model architecture (The "Oven")
# Note: The parameters must perfectly match what you used during training!
loaded_model = SimpleNN(
    input_size=X_train.shape[1],
    hidden_size=8,
    num_layers=2,
    dropout=0.1
)

# 2. Load the saved weights (The "Recipe") into the empty model
loaded_model.load_state_dict(torch.load("titanic_best_model.pth"))

# 3. CRITICAL: Set the model to evaluation mode
loaded_model.eval()

print("Model loaded and ready to make predictions!")